In [155]:
from Bio import SeqIO
from Bio.Seq import Seq
import csv
import gzip
import itertools
import pandas as pd
import numpy as np

Here we read all the sequence indeces for the forward and reverse reads

In [156]:
forward_dict = SeqIO.to_dict(SeqIO.parse("FW_primers.fa", "fasta"))
forward_dict = {k:str(v.seq) for k, v in forward_dict.items()}
reverse_dict = SeqIO.to_dict(SeqIO.parse("RV_primers.fa", "fasta"))
reverse_dict = {k:str(v.seq.reverse_complement()) for k, v in reverse_dict.items()}

Now, we read all the plasmid sequences

In [157]:
plasmids = {}
with open('Plasmid_ID_Correct_position_Riley.csv', 'r') as csvfile:
    table_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in table_reader:
        plasmids[row["Barcode name"]]= row["ID plasmid sequence"]

In [158]:
table_reader

Let's iterate the two sequence files and produce the table with the distances

In [159]:
def find_hash_position(sequence, tags):
    #print("________")
    #print(sequence)
    #print("________")
    for key, value in tags.items():
    #    print(value)
        position = sequence.find(value)
        if position != -1:
            return (key, position)
    return ("none", -1)

fw_f = "C:/Users/Lab/Documents/Riley/240528_PlasmidID_analysis/MasterScripts_240528/240528_MergedFastq/Riley16_P4_S24_R_001_mergedUnpaired.fastq.gz"
distances_f = "distances_merge.txt"
with gzip.open(fw_f, "rt") as r1,open(distances_f,"w") as f3:
    f3.write("fw_name,fw_pos,rev_name,rev_pos,pl_name,pl_pos\n")
    for fw in SeqIO.parse(r1, "fastq") :
        #print(fw)
        str_seq = str(fw.seq)
        fw_name, fw_pos = find_hash_position(str_seq, forward_dict)
        rv_name, rv_pos = find_hash_position(str_seq, reverse_dict)
        pl_name, pl_pos = find_hash_position(str_seq, plasmids)
        f3.write(",".join([fw_name, str(fw_pos), rv_name, str(rv_pos), pl_name,str(pl_pos)])) 
        f3.write("\n")
    
    

In [160]:
df = pd.read_csv(distances_f)
df

,fw_name,fw_pos,rev_name,rev_pos,pl_name,pl_pos
0,10_F,0,F_R,54,rbc413,42
1,6_F,0,A_R,56,rbc413,44
2,12_F,0,A_R,56,rbc413,44
3,12_F,0,F_R,56,rbc404,44
4,2_F,0,G_R,55,rbc413,43
...,...,...,...,...,...,...
43668,9_F,0,F_R,56,rbc413,44
43669,3_F,0,G_R,56,rbc413,44
43670,3_F,0,G_R,56,rbc413,44
43671,10_F,0,F_R,54,rbc413,42


In [161]:
summ=df[['fw_name', 'rev_name', 'pl_name', 'pl_pos']].groupby(['fw_name', 'rev_name', 'pl_name']).agg( ['count','mean'])

In [162]:
summ.to_csv("Tc20_C_RW.csv", sep=',')

In [163]:
summ

pl_pos           
                          count       mean
fw_name rev_name pl_name                  
10_F    A_R      none         4  -1.000000
                 rbc104       7  42.000000
                 rbc402      12  42.000000
                 rbc403       2  42.000000
                 rbc404       1  42.000000
...                         ...        ...
none    none     rbc104       1  44.000000
                 rbc402       1  42.000000
                 rbc403       1  43.000000
                 rbc410       1  43.000000
                 rbc413      24  43.041667

[680 rows x 2 columns]